# Analysis of NordStream leakages

Run three scenarios
- ssp245 as the counterfactual
- a scenario with an additional pulse of 250 ktCH4 in 2022, which is the estimated total methane release from the NordStream leak (source: https://www.euronews.com/green/2022/09/28/nord-stream-russian-gas-pipe-leaks-could-have-an-unprecedented-environmental-impact)
- a scenario where this 200 ktCH4 would have been burned and released as CO2.

Run scenarios in FaIR v2.1 using AR6 calibrated parameters in deterministic climate mode.

## Required imports

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch
import xarray as xr

from fair import FAIR
from fair.io import read_properties
from fair.interface import initialise, fill

## Set up FaIR and scenarios

In [ ]:
# initialise
f = FAIR()

# declare time horizon of simulation
f.define_time(1750, 2100, 1)

# declare scenario names and add to FaIR
scenarios = ['ssp245', 'leak', 'burned_as_co2']
f.define_scenarios(scenarios)

# load up default list of species and their properties
species, properties = read_properties()

# declare species and properties
f.define_species(species, properties)

### Obtain AR6 calibrations

In [ ]:
fair_params_df = pd.read_csv('../data/fair2.1.0_ar6_calibration_ebm3_20220929.csv', index_col=0)
fair_params_df

In [ ]:
# declare config names (we use the probabilistic draw index)
f.define_configs(list(fair_params_df.index))

### Create emissions and natural forcing time series

In [ ]:
# Create arrays of emissions, etc. in FaIR
f.allocate()

In [ ]:
# Grab pre-prepared SSP emissions, which are from RCMIP 
# (original source: Nicholls & Lewis 2021, https://zenodo.org/record/4589756#.YzVyonZByUk)
ssp_emissions = xr.load_dataarray('../data/ssp_emissions_fair2.1.nc')

In [ ]:
# Copy SSP2-4.5 emissions to our three scenarios
for scenario in scenarios:
    f.emissions.loc[
        dict(scenario=scenario)
    ] = ssp_emissions.loc[
        dict(
            scenario='ssp245', 
            timepoints=slice(1750.5, 2100), 
            config='unspecified'
        )
    ].drop_vars('config').expand_dims(dim='config', axis=1)

In [ ]:
# Add 0.2 MtCH4 to the emissions time series in 2022 for the "leak" scenario
f.emissions.loc[
    dict(
        specie='CH4', 
        timepoints=2022.5, 
        scenario='leak'
    )
] = f.emissions.loc[
    dict(
        specie='CH4', 
        timepoints=2022.5,
        scenario='leak'
    )
] + 0.25

In [ ]:
# Add the equivalent as CO2 fossil in the "burned_as_co2" scenario
# Molecular weight CO2:CH4 = 44.009/16.043
# Divide by 1000 to convert native units of MtCH4 in FaIR to GtCO2
f.emissions.loc[
    dict(
        specie='CO2 FFI', 
        timepoints=2022.5,
        scenario='burned_as_co2'
    )
] = f.emissions.loc[
    dict(
        specie='CO2 FFI', 
        timepoints=2022.5, 
        scenario='burned_as_co2'
    )
] + 0.25 / 1000 * 44.009 / 16.043

In [ ]:
# get solar and volcanic forcing from AR6
ar6_forcing_df = pd.read_csv('../data/table_A3.3_historical_ERF_1750-2019_best_estimate.csv', index_col=0)
ar6_forcing_df

In [ ]:
# Ramp volcanic down to zero over a decade, following CMIP6 convention
volcanic_forcing = np.zeros(351)
volcanic_forcing[:270] = ar6_forcing_df['volcanic'].values
volcanic_forcing[269:281] = np.linspace(1, 0, 12) * volcanic_forcing[269]

In [ ]:
# Use a zero solar amplitude post-2019, to isolate anthropogenic warming signal
solar_forcing=np.zeros(351)
solar_forcing[:270] = ar6_forcing_df['solar'].values

In [ ]:
# Put volcanic forcing into FaIR
fill(
    f.forcing, 
    volcanic_forcing[:, None, None] * fair_params_df.loc[:, 'scale Volcanic'].values[None, None, :], 
    specie='Volcanic'
)

In [ ]:
# Put solar forcing into FaIR
trend_shape = np.ones(351)
trend_shape[:271] = np.linspace(0, 1, 271)

fill(f.forcing, 
     solar_forcing[:, None, None] * 
     fair_params_df.loc[:, 'solar_amplitude'].values.squeeze() + 
     trend_shape[:, None, None] * fair_params_df.loc[:, 'solar_trend'].values.squeeze(),
     specie='Solar'
)

### Set up climate and species-level response

In [ ]:
# Get default species configs
f.fill_species_configs()

# Climate response
fill(f.climate_configs['ocean_heat_capacity'], fair_params_df.loc[:,'c1':'c3'])
fill(f.climate_configs['ocean_heat_transfer'], fair_params_df.loc[:,'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[:,'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], fair_params_df.loc[:,'gamma'])
fill(f.climate_configs['stochastic_run'], False)

# carbon cycle
fill(f.species_configs['iirf_0'], fair_params_df.loc[:, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], fair_params_df.loc[:, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], fair_params_df.loc[:, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], fair_params_df.loc[:, 'rT'].values.squeeze(), specie='CO2')

# aerosol direct
for specie in ['BC', 'CH4', 'N2O', 'NH3', 'NOx', 'OC', 'Sulfur', 'VOC', 'Equivalent effective stratospheric chlorine']:
    fill(f.species_configs['erfari_radiative_efficiency'], fair_params_df.loc[:, f'ari {specie}'].values.squeeze(), specie=specie)

# aerosol indirect
fill(f.species_configs['aci_scale'], fair_params_df.loc[:, 'beta'].values.squeeze())
fill(f.species_configs['aci_shape'], fair_params_df.loc[:, 'shape_so2'].values.squeeze(), specie='Sulfur')
fill(f.species_configs['aci_shape'], fair_params_df.loc[:, 'shape_bc'].values.squeeze(), specie='BC')
fill(f.species_configs['aci_shape'], fair_params_df.loc[:, 'shape_oc'].values.squeeze(), specie='OC')

# ozone
for specie in ['CH4', 'N2O', 'Equivalent effective stratospheric chlorine', 'CO', 'VOC', 'NOx']:
    fill(f.species_configs['ozone_radiative_efficiency'], fair_params_df.loc[:, f'o3 {specie}'], specie=specie)

# methane lifetime baseline
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# forcing scaling
for specie in ['CH4', 'N2O', 'Stratospheric water vapour', 'Contrails', 'Light absorbing particles on snow and ice', 'Land use']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[:, f'scale {specie}'].values.squeeze(), specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CHCl3', 'CH2Cl2', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C3F8', 'c-C4F8', 'C4F10', 'C5F12', 'C6F14', 'C7F16', 'C8F18', 'NF3', 'SF6', 'SO2F2',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    fill(f.species_configs['forcing_scale'], fair_params_df.loc[:, 'scale minorGHG'].values.squeeze(), specie=specie)

# Scale CO2 forcing based on its 4xCO2 calibration
calibrated_f4co2_mean = fair_params_df.loc[:,'F_4xCO2'].values.mean()
fill(
    f.species_configs['forcing_scale'], 
    1 + 0.561*(calibrated_f4co2_mean - fair_params_df.loc[:,'F_4xCO2'].values)/calibrated_f4co2_mean,
    specie='CO2'
)

# tune down volcanic efficacy
fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs['baseline_concentration'], 
    fair_params_df.loc[:, 'co2_concentration_1750'].values.squeeze(),
    specie='CO2'
)

# Use interactive methane lifetime
f.ch4_method='Thornhill2021'

In [ ]:
# set initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.airborne_emissions, 0)
initialise(f.cumulative_emissions, 0)

## Run FaIR

In [ ]:
# run in parallel
f.run()

## Analysis

In [ ]:
# make directory for plots
os.makedirs('../plots', exist_ok=True)

### Probabalistic temperature projections in each scenario

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(12, 4))

for i in range(3):
    ax[i].fill_between(
        f.timebounds, 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 5, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].fill_between(
        f.timebounds, 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 16, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].plot(
        f.timebounds, 
        np.median(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i].set_xlim(1750,2100)
    ax[i].set_ylim(-1, 4)
    ax[i].axhline(0, color='k', ls=":", lw=0.5)
    ax[i].set_title(scenarios[i])
pl.suptitle('Temperature anomaly')
#fig.tight_layout()

### Differences compared to baseline

In [ ]:
pl.plot(
    f.timebounds,
    1e6 * (
        (np.median(f.temperature[:, 1, :, 0]-f.temperature[100:151, 1, :, 0].mean(axis=0), axis=1)) - 
        (np.median(f.temperature[:, 0, :, 0]-f.temperature[100:151, 0, :, 0].mean(axis=0), axis=1))
    ),
    label = 'NordStream leakage'
)
pl.plot(
    f.timebounds,
    1e6 * (
        (np.median(f.temperature[:, 2, :, 0]-f.temperature[100:151, 2, :, 0].mean(axis=0), axis=1)) - 
        (np.median(f.temperature[:, 0, :, 0]-f.temperature[100:151, 0, :, 0].mean(axis=0), axis=1))
    ),
    label = 'If leaked methane was burned as CO$_2$'
)
pl.xlim(2020, 2100)
pl.ylabel('$\mu$ K (millionths °C) above baseline')
pl.title('NordStream pipeline leakage (250 ktCH$_4$) climate impacts')
pl.savefig('../plots/NordStream.png')
pl.legend()

### Check that individual scenarios are smooth

In [ ]:
pl.plot(
    f.timebounds,
    1e6 * (
        (f.temperature[:, 1, :, 0]-f.temperature[100:151, 1, :, 0].mean(axis=0)) - 
        (f.temperature[:, 0, :, 0]-f.temperature[100:151, 0, :, 0].mean(axis=0))
    ),
    label = 'NordStream leakage'
);
pl.xlim(2020, 2100)